<a href="https://colab.research.google.com/github/Seong-JiHyeon/NLTK/blob/master/Back%20off%20n-gram%20pos%20tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

True

태거(tagger)는 nltk 5.5절의 back off n-gram 태거에 기반하여, 브라운 전체코퍼스(category 제한하지 말 것)를 트레이닝하여  개발

tagset은 default tagset을 사용하면 되나, 다른 태그셋도 사용가능

발견되는 문제점이 있으면, 성능을 향상 시킬 수 있는 방안이 있는지에 대한 제안도 간단히 기술해 줄 것.

전체 정확도를 기술해 주고, 여러개의 문장을 적용한 결과를 예시로 올리기

In [0]:
from nltk.corpus import *
brown_tagged_sents = brown.tagged_sents()

In [0]:
data = []
for tag_sent in brown_tagged_sents:
  for tag_word in tag_sent:
    data.append(tag_word)

In [0]:
data

[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN'),
 ("Atlanta's", 'NP$'),
 ('recent', 'JJ'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', '``'),
 ('no', 'AT'),
 ('evidence', 'NN'),
 ("''", "''"),
 ('that', 'CS'),
 ('any', 'DTI'),
 ('irregularities', 'NNS'),
 ('took', 'VBD'),
 ('place', 'NN'),
 ('.', '.'),
 ('The', 'AT'),
 ('jury', 'NN'),
 ('further', 'RBR'),
 ('said', 'VBD'),
 ('in', 'IN'),
 ('term-end', 'NN'),
 ('presentments', 'NNS'),
 ('that', 'CS'),
 ('the', 'AT'),
 ('City', 'NN-TL'),
 ('Executive', 'JJ-TL'),
 ('Committee', 'NN-TL'),
 (',', ','),
 ('which', 'WDT'),
 ('had', 'HVD'),
 ('over-all', 'JJ'),
 ('charge', 'NN'),
 ('of', 'IN'),
 ('the', 'AT'),
 ('election', 'NN'),
 (',', ','),
 ('``', '``'),
 ('deserves', 'VBZ'),
 ('the', 'AT'),
 ('praise', 'NN'),
 ('and', 'CC'),
 ('thanks', 'NNS'),
 ('of', 'IN'),
 ('the', '

In [0]:
tag_fd = nltk.FreqDist(tag for (word, tag) in data)
tag_fd.most_common(10)

[('NN', 152470),
 ('IN', 120557),
 ('AT', 97959),
 ('JJ', 64028),
 ('.', 60638),
 (',', 58156),
 ('NNS', 55110),
 ('CC', 37718),
 ('RB', 36464),
 ('NP', 34476)]

-> 가장 많이 사용된 품사는 NN으로 default tagging의 조건으로 사용하도록 한다

In [0]:
# train, test split
size= int(len(brown_tagged_sents)*0.9)
train_sents= brown_tagged_sents[:size]
test_sents = brown_tagged_sents[size:]

In [0]:
tagger1 = nltk.DefaultTagger('NN')
tagger2 = nltk.UnigramTagger(train_sents, backoff=tagger1)
tagger3 = nltk.BigramTagger(train_sents, backoff=tagger2)
tagger4 = nltk.TrigramTagger(train_sents, backoff=tagger3)

In [0]:
tagger5 = nltk.NgramTagger(4,train_sents, backoff=tagger4)

In [0]:
tagger1.evaluate(test_sents)

0.10674545797447664

In [0]:
tagger2.evaluate(test_sents)

0.8912742817627459

In [0]:
tagger3.evaluate(test_sents)

0.9125751765470128

In [0]:
tagger4.evaluate(test_sents)

0.9130466670857693

In [0]:
tagger5.evaluate(test_sents)

0.9122398943861193

-> tagger5에서 오히려 성능이 조금 떨어짐



---



In [0]:
from nltk.tokenize import *
for i in range(5):
  sent = input("Enter a sentence: ")
  tokens = word_tokenize(sent)
  result = ""
  for (word,tag) in tagger4.tag(tokens):
    result = result +tag+" "
  print("POS Tagging result: ", result)
  print("")

Enter a sentence: time flies like an arrow
POS Tagging result:  NN NNS VB AT NN 

Enter a sentence: fruit flies like a banana
POS Tagging result:  NN NNS VB AT NN 

Enter a sentence: I really love my dog.
POS Tagging result:  PPSS RB VB PP$ NN . 

Enter a sentence: I'm really happy because I finished my assignment
POS Tagging result:  PPSS NN RB JJ RB PPSS VBD PP$ NN 

Enter a sentence: No sweat, no sweet
POS Tagging result:  AT NN , AT JJ 



In [0]:
nltk.help.upenn_tagset('.*')

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [0]:
tagger4.tag(['am'])

[('am', 'BEM')]

In [0]:
tagger4.tag(['is'])

[('is', 'BEZ')]

In [0]:
tagger4.tag(['are'])

[('are', 'BER')]

1. flies(NNS: 동사가 아닌 복수형 명사로 인식됨) like(VB: 전치사가 아닌 동사로 인식됨) 

2. 위와 반대로 잘 인식됨

3. 특이사항 없음

4. 'm이 NN으로 인식됨. am이 잘못 tagging 됨

5. 특이사항 없음

In [0]:
wnl = nltk.WordNetLemmatizer()
for i in range(5):
  sent = input("Enter a sentence: ")
  tokens = word_tokenize(sent)
  lem_tokens=[]
  for token in tokens:
    lem_tokens += wnl.lemmatize(token)
  result = ""
  for (word,tag) in tagger4.tag(tokens):
    result = result +tag+" "
  print("POS Tagging result: ", result)
  print("")

Enter a sentence: time flies like an arrow
POS Tagging result:  NN NNS VB AT NN 

Enter a sentence: fruit flies like a banana
POS Tagging result:  NN NNS VB AT NN 

Enter a sentence: I'm really happy because I finished my assignment
POS Tagging result:  PPSS NN RB JJ RB PPSS VBD PP$ NN 

Enter a sentence: really not working...
POS Tagging result:  RB * VBG NN 

Enter a sentence: what's that?
POS Tagging result:  WDT NN WPS . 



-> 특수한 경우(관용적 문구)는 추가적으로 데이터를 구축하여 학습시키는 과정이 필요함

-> Regular expression을 활용하여 (‘s, ‘m …)에 대한 규칙을 추가하여 학습하는 과정 필요함